In [1]:
import pandas as pd
import numpy as np
import seaborn as sbn

df = pd.read_csv("Advertising.csv")
df = df.iloc[:,1:len(df)]
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [5]:
X = df.drop('sales',axis=1)
y = df[["sales"]]

In [6]:
y.head()

,sales
0,22.1
1,10.4
2,9.3
3,18.5
4,12.9


In [4]:
X.head()

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4


**Statsmodel ile model kurmak**

In [8]:
import statsmodels.api as sm

In [9]:
model = sm.OLS(y,X).fit()

In [10]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              3566.
Date:                Fri, 01 Sep 2023   Prob (F-statistic):                   2.43e-171
Time:                        00:27:24   Log-Likelihood:                         -423.54
No. Observations:                 200   AIC:                                      853.1
Df Residuals:                     197   BIC:                                      863.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0538      0.001     40.507      0.000       0.051       0.056
radio          0.2222      0.009     23.595      0.000       0.204       0.241
newspaper      0.0168      0.007      2.517      0.013       0.004       0.030
==============================================================================
Omnibus:                        5.982   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.050   Jarque-Bera (JB):                7.039
Skew:                          -0.232   Prob(JB):                       0.0296
Kurtosis:                       3.794   Cond. No.                         12.6
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
##Scikit-learn ile model oluşturma

from sklearn.linear_model import LinearRegression
model  = LinearRegression().fit(X,y)

In [12]:
model.intercept_

array([2.93888937])

In [13]:
model.coef_

array([[ 0.04576465,  0.18853002, -0.00103749]])

*katsayıların farklı olması, ilgili kütüphanelerin kullanmış oldukları parametre tahmin yöntemleri ile ilgilidir.*

###### Tahmin

Sales = 2.94 + TV * 0.04 + radio * 0.19 - newspaper * 0.001

30 birim TV, 10 birim radio, 40 birim gazete

In [16]:
2.94 + 30 * 0.04 + 10 * 0.19 - 40 * 0.001

5.999999999999999

In [17]:
yeni_veri = [[30],[10],[40]]

In [18]:
yeni_veri = pd.DataFrame(yeni_veri).T

In [19]:
yeni_veri

,0,1,2
0,30,10,40


In [20]:
model.predict(yeni_veri)

C:\Users\menes\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[6.15562918]])

In [21]:
from sklearn.metrics import mean_squared_error

In [23]:
y.head()

,sales
0,22.1
1,10.4
2,9.3
3,18.5
4,12.9


In [24]:
model.predict(X)[0:10]

array([[20.52397441],
       [12.33785482],
       [12.30767078],
       [17.59782951],
       [13.18867186],
       [12.47834763],
       [11.72975995],
       [12.12295317],
       [ 3.72734086],
       [12.55084872]])

In [27]:
MSE = mean_squared_error(y,model.predict(X))
MSE

2.784126314510936

In [28]:
RMSE = np.sqrt(MSE)
RMSE

1.6685701407225697

### Model tuning (Model doğrulama)

In [29]:
# sınama seti
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=99)

In [31]:
X_train.head()

,TV,radio,newspaper
16,67.8,36.6,114.0
51,100.4,9.6,3.6
97,184.9,21.0,22.0
164,117.2,14.7,5.4
71,109.8,14.3,31.7


In [33]:
y_train.head()

,sales
16,12.5
51,10.7
97,15.5
164,11.9
71,12.4


In [34]:
model = LinearRegression().fit(X_train,y_train)

In [36]:
## eğitim hatası
np.sqrt(mean_squared_error(y_train,model.predict(X_train)))

1.7236824822650751

In [37]:
## test hatası
np.sqrt(mean_squared_error(y_test,model.predict(X_test)))

1.4312783138301635

In [38]:
## k-katlı cross validation
from sklearn.model_selection import cross_val_score

In [41]:
## cv = kat sayısı
cross_val_score(model,X_train,y_train,cv=10,scoring="neg_mean_squared_error")

array([-2.1019073 , -2.48953197, -3.09704214, -2.34694216, -3.68175761,
       -1.8691401 , -3.18173007, -4.1927349 , -2.17128376, -8.03821974])

In [42]:
## cv mse
np.mean(-cross_val_score(model,X_train,y_train,cv=10,scoring="neg_mean_squared_error"))

3.3170289742341255

In [43]:
## cv rmse
np.sqrt(np.mean(-cross_val_score(model,X_train,y_train,cv=10,scoring="neg_mean_squared_error")))

1.8212712522395245

In [44]:
np.sqrt(np.mean(-cross_val_score(model,X,y,cv=10,scoring="neg_mean_squared_error")))
## veri olarak X-y gönderildi. tüm veri seti tek parça halinde gönderilmiş oldu.

1.7492763126843378